In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2400)

In [15]:
x[:,:2]

matrix([[ 1249.41244558,  1255.45433719],
        [ 1229.34069321,  1226.72965574],
        [ 1530.37968128,  1511.08237574],
        [ 1472.98509888,  1511.13898489],
        [ 1558.81095259,  1562.6866878 ],
        [ 1623.29312469,  1625.54842903],
        [ 1327.02213278,  1298.41741842],
        [ 1303.27834728,  1274.44082988],
        [ 1734.56321675,  1718.82562358],
        [ 1615.73983028,  1686.31395525],
        [ 1474.12155147,  1543.70184845],
        [  875.19055874,   890.7326401 ],
        [ 1276.09814154,  1376.00757579],
        [ 1351.54381935,  1315.01067064],
        [ 1228.45631572,  1308.25627423],
        [  950.33044993,   950.52232864],
        [ 1286.4776891 ,  1285.77044483],
        [ 1288.15644082,  1258.62135467],
        [ 1708.03291644,  1677.43948194],
        [ 1585.86710296,  1561.76945484],
        [ 1292.81218273,  1310.93292643],
        [ 1495.16837467,  1509.98690213],
        [ 1289.79221808,  1290.04729188],
        [ 1183.20513023,  1252.342

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 3e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 27
 AA' NZ     : 3.510e+02
 Factor NZ  : 3.780e+02
 Factor Ops : 6.930e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.87950850e+07  0.00000000e+00  6.61e+03 1.39e+03  1.00e+06     0s
   1  -1.32710907e+07 -8.28740542e+06  9.80e+02 2.06e+02  2.13e+05     0s
   2  -6.72566979e+06 -7.78770784e+06  3.53e+01 7.41e+00  2.79e+04     0s
   3  -6.71266719e+06 -6.83537399e+06  1.51e-01 3.17e-02  2.2

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.5808825630979428e-12,
 1688.1845106479782,
 1.7553998940341274e-12,
 522.5683688483213,
 482.64578751562146,
 12.00358099046693,
 5.297252478478197e-10,
 912.1029043327986,
 7.704909567558235e-12,
 1378.1866028118625,
 7.704909567542726e-12,
 7.704909567550687e-12,
 7.704909567514808e-12,
 7.749093015683718e-12,
 649.8599538939275,
 0.008159001522745513,
 159.25955998690435,
 666.6380122424335,
 588.5871295097495,
 7.422897292828736,
 196.90888475725473,
 1.8437622404361286e-05,
 1300.9157551741807,
 9.501820916150638e-07,
 1.3384584484349027,
 741.4843088559168,
 0.07411546565038885,
 1102.0562166725651,
 9.500134051479187e-07,
 9.499951789156954e-07,
 9.500867396695608e-07,
 0.043478822175769594,
 301.8689902111281,
 0.7564810911491082,
 1036.7588687909576,
 219.4783326251461,
 0.005413651736100145,
 0.013462507438695814,
 933.1404887586515,
 0.21652707698538287,
 816.4508533058352,
 15.113075090907302,
 0.28627629290518897,
 0.008954845333959405,
 372.6516777231023,
 94.040476821